In [1]:
import numpy as np
from keras.layers import Embedding
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


# Model 1: Training Custom Embedding Vectors

In [2]:
docs = ['Well done!','Good work','Great effort','nice work','Excellent!',
        'Weak','Poor effort!','not good','poor work','Could have done better.']

In [3]:
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [4]:
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[23, 36], [3, 22], [18, 38], [1, 22], [48], [41], [33, 38], [16, 3], [33, 22], [40, 36, 36, 37]]


In [5]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[23 36  0  0]
 [ 3 22  0  0]
 [18 38  0  0]
 [ 1 22  0  0]
 [48  0  0  0]
 [41  0  0  0]
 [33 38  0  0]
 [16  3  0  0]
 [33 22  0  0]
 [40 36 36 37]]


In [6]:
type(padded_docs), padded_docs.shape

(numpy.ndarray, (10, 4))

In [7]:
output_vector_space = 8

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim = output_vector_space, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(padded_docs, labels, epochs=50, verbose=0)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


In [10]:
def preprocess_input(text):
    vocab_size = 50
    encoded_docs = [one_hot(text, vocab_size)]
    max_length = 4
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs

In [11]:
def predict_intent(text):
    custom_input = preprocess_input(text)
    model_prediction = model.predict(custom_input)[0][0]
    prediction = 'Positive' if model_prediction >= 0.5 else 'Negative'
    return prediction

In [12]:
predict_intent('good effort'), predict_intent('not great')

('Positive', 'Negative')

# Model 2: Using pre-trained weights

In [13]:
docs = ['Well done!','Good work','Great effort','nice work','Excellent!',
        'Weak','Poor effort!','not good','poor work','Could have done better.']

In [14]:
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
vocab_size = len(tokenizer.word_index) + 1

In [16]:
encoded_docs = tokenizer.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [17]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [18]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)

NameError: name 'embedding_matrix' is not defined